In [73]:
from __future__ import absolute_import, division, print_function, unicode_literals
import pandas as pd
import tensorflow as tf
import csv
from google.colab import files
import io

In [ ]:
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv')
eval = {'survived': [0],
        'sex': [input("What is your gender " + str(feature_columns[0][1]) + " ? " )],
        'age': [int(input("What is your age " + str(feature_columns[7][1]) + " ? "))],
        'n_siblings_spouses': [int(input("How many siblings " + str(feature_columns[1][1]) + " ? "))],
        'parch': [int(input("What is your parch " + str(feature_columns[2][1]) + " ? "))],
        'fare': [int(input("What is the fare " + str(feature_columns[8][1]) + " ? "))],
        'class': [input("What is your class " + str(feature_columns[3][1]) + " ? ")],
        'deck': [input("What is your deck " + str(feature_columns[4][1]) + " ? ")],
        'embark_town': [input("What is your destination " + str(feature_columns[5][1]) + " ? ")],
        'alone': [input("Are you travelling alone " + str(feature_columns[6][1]) + " ? ")]}
dfeval = pd.DataFrame(eval) 
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']
feature_columns = []
for feature_name in CATEGORICAL_COLUMNS:
  vocabulary = dftrain[feature_name].unique()
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))
for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))
def make_input_fn(data_df, label_df, num_epochs=15, shuffle=True, batch_size=32):
  def input_function(): 
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000) 
    ds = ds.batch(batch_size).repeat(num_epochs)  
    return ds 
  return input_function 
train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)
linear_est.train(train_input_fn)
result = list(linear_est.predict(eval_input_fn))
print("Chance of your survival: " + str(result[0]['probabilities'][1] * 100) + "%") 